In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://hcahealthcare.com/locations/?state=TN"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html5lib')
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en-us">
 <head>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <!-- To kick IE out of quirks mode so Respond.js works -->
  <meta content="initial-scale=1.0, maximum-scale=1.0, user-scalable=no, width=device-width" name="viewport"/>
  <meta charset="utf-8"/>
  <!-- <meta name="HandheldFriendly" content="true"/> -->
  <!-- BlackBerry? -->
  <!-- <meta name="MobileOptimized" content="width" /> -->
  <!-- IE Mobile? -->
  <title>
   Locations | HCA Healthcare
  </title>
  <link href="https://hcahealthcare.com/locations/index.dot" rel="canonical"/>
  <meta content="Locations" property="og:title"/>
  <meta content="Website" property="og:type"/>
  <meta content="https://hcahealthcare.com/locations/index.dot" property="og:url"/>
  <meta content="https://hcahealthcare.com/contentAsset/raw-data/f430917e-9fa1-4f1a-8dc7-d652c0b00e71/fullColorLogo" property="og:image"/>
  <meta content="HCA Healthcare" property="og:site_name"/>
  <meta content=

In [2]:
# Does it see what we need?
'Parkridge Medical Center' in str(soup)

True

In [3]:
# Backup
from pathlib import Path

saved_path = Path('saved_html/HCA_TN/HCA_TN_hosps.html')
with saved_path.open('r') as f:
    lines = f.readlines()
html = '\n'.join(lines)

soup = BeautifulSoup(html, 'html5lib')

In [4]:
# Does it see what we need?
'Parkridge Medical Center' in str(soup)

True

## Objective: 
### Find all facilities of major hospital chains

##### Fields to grab:
+ Hospital Name
+ Address
+ City
+ State
+ Zip
+ Phone Number
+ Website

In [5]:
import re 
regex = re.compile('location[0-9]+') # equivilent to 'location\d+'

location_section = soup.find('div', id='locations-map-view')
results = location_section.find_all('div', attrs={'data-id': regex}) #, 'style': 'display: block;'})
len(results)

14

## Strategy
#### Break section down to one card and tailor text extraction for each peice of data
+ Each cards section might have several peices
+ Continue to break each 

In [6]:
%%html
# <div class="location-item location-detail no-er-time feed-loading" id="location1" data-id="location1" data-name="Parkridge East Hospital" data-type="ER" data-showlocation="true" itemprop="location" itemscope="" itemtype="https://schema.org/Place" data-distance="" style="display: block;">
#    <div class="card">
#       <div class="row">
#          <div class="col-sm-3">
#             <img class="building-number pull-left  hidden-xs" src="//core.secure.ehc.com/src/apps/maps-and-locations/images/pins/flat/4a4a4a/circle/1.png" data-color="4a4a4a">
#             <img class="building-icon map-small-image circle img-loaded" src="//hcahealthcare.com/util/images/location-images/parkridge-east-hospital.jpg" data-img-src="//hcahealthcare.com/util/images/location-images/parkridge-east-hospital.jpg" alt="">
#          </div>
#          <div class="col-sm-9">
#             <div class="map-sidebar-info-block">
               <p class="location-title">
                  <a href="http://parkridgeeasthospital.com/" data-open="popup" itemprop="name">Parkridge East Hospital</a>
                  <br>
                  <span class="location-distance"> </span>
               </p>
            
               <address itemprop="address" itemscope="" itemtype="https://schema.org/PostalAddress" data-address="941 Spring Creek Rd, Chattanooga, TN 37412" data-id="12972112-bef1-4fbd-9377-51f95f857c7a">
                  <span itemprop="streetAddress">941 Spring Creek Rd</span><br><span itemprop="addressLocality">Chattanooga</span>,&nbsp;<span itemprop="addressRegion">TN</span>&nbsp;<span itemprop="addressRegion">37412</span><br><span class="hidden-xs" itemprop="telephone">Phone: (423) 894-7870</span> 
               </address>
#                <meta itemprop="geo" content="34.998737,-85.218966">
#                <p class="listing-wait-distance">
#                </p>
#                <p class="links hidden-xs">
#                   <a href="http://parkridgeeasthospital.com/" class="location-page" target="_blank" rel="noopener noreferrer">Visit Site</a> |
#                   <a href="/locations/?directions=Parkridge East Hospital" onclick="get_directions_to('location1'); return false;" "="" class="directions">Get Directions</a>
#                </p>
#                <div class="links visible-xs">
#                   <div class="row">
#                      <div class="col-xs-6">
#                         <a class="btn btn-outline-primary btn-block" href="http://parkridgeeasthospital.com/" target="_blank" rel="noopener noreferrer">Visit Site</a>
#                      </div>
#                      <div class="col-xs-6">
#                         <a class="btn btn-outline-primary btn-block" href="tel:4238947870">Call Now</a>
#                      </div>
#                      <div class="col-xs-12">
#                         <a href="/locations/?directions=Parkridge East Hospital" onclick="get_directions_to('location1'); return false;" "="" class="directions btn btn-primary btn-block">Get Directions</a>
#                      </div>
#                   </div>
#                </div>
#             </div>
#          </div>
#       </div>
#    </div>
# </div>

## Address Info

In [8]:
location = results[0]

address = location.find('address')
address

<address data-address="941 Spring Creek Rd, Chattanooga, TN 37412" data-id="12972112-bef1-4fbd-9377-51f95f857c7a" itemprop="address" itemscope="" itemtype="https://schema.org/PostalAddress">
														<span itemprop="streetAddress">941 Spring Creek Rd</span><br/><span itemprop="addressLocality">Chattanooga</span>, <span itemprop="addressRegion">TN</span> <span itemprop="addressRegion">37412</span><br/><span class="hidden-xs" itemprop="telephone">Phone: (423) 894-7870</span> 
													</address>

In [9]:
%%html
<address itemprop="address" itemscope="" itemtype="https://schema.org/PostalAddress" data-address="941 Spring Creek Rd, Chattanooga, TN 37412" data-id="12972112-bef1-4fbd-9377-51f95f857c7a">
<span itemprop="streetAddress">941 Spring Creek Rd</span><br>
<span itemprop="addressLocality">Chattanooga</span>,&nbsp;
<span itemprop="addressRegion">TN</span>&nbsp;
<span itemprop="addressRegion">37412</span><br>
<span class="hidden-xs" itemprop="telephone">Phone: (423) 894-7870</span> 
</address>

In [32]:
# Address
street_address = address.find('span', attrs={'itemprop': 'streetAddress'})
street_address

<span itemprop="streetAddress">941 Spring Creek Rd</span>

In [11]:
# City
address.find('span', attrs={'itemprop': 'addressLocality'}).text

'Chattanooga'

In [12]:
# State and Zip, multiple properties with the same name
regions = address.find_all('span', attrs={'itemprop': 'addressRegion'})
for item in regions:
    if item.text.isdigit():
        zip_code = item.text
    else:
        state_code = item.text
        
zip_code, state_code

('37412', 'TN')

In [13]:
# Phone Numbers 1
phone_field = address.find('span', attrs={'itemprop': 'telephone'}).text
phone_field

'Phone: (423) 894-7870'

In [14]:
''.join([d for d in phone_field if d.isdigit()])

'4238947870'

In [12]:
# Phone Numbers 2
# source: http://www.diveintopython.net/regular_expressions/phone_numbers.html
# (\d{3})     # area code is 3 digits (e.g. '800')
# \D*         # optional separator is any number of non-digits
# (\d{3})     # trunk is 3 digits (e.g. '555')
# \D*         # optional separator
# (\d{4})     # rest of number is 4 digits (e.g. '1212')

phone_pattern = re.compile('(\d{3})\D*(\d{3})\D*(\d{4})')
phone_field = address.find('span', attrs={'itemprop': 'telephone'}).text

matches = re.search(phone_pattern, phone_field)
''.join(matches.groups())

'4238947870'

In [21]:
def extract_address(soupy_html_address):
    """
        You should probably add default values and/or missing data logging.
    """
    street_address = soupy_html_address.find('span', attrs={'itemprop': 'streetAddress'}).text
    city = soupy_html_address.find('span', attrs={'itemprop': 'addressLocality'}).text
    
    regions = soupy_html_address.find_all('span', attrs={'itemprop': 'addressRegion'})
    for item in regions:
        if item.text.isdigit():
            zip_code = item.text
        else:
            state_code = item.text

    phone_pattern = re.compile('(\d{3})\D*(\d{3})\D*(\d{4})')
    phone_field = soupy_html_address.find('span', attrs={'itemprop': 'telephone'}).text

    matches = re.search(phone_pattern, phone_field)
    phone_number = ''.join(matches.groups())
    
    return {
        'address': street_address,
        'city': city,
        'state_code': state_code,
        'zip_code': zip_code,
        'phone': phone_number
    }

extract_address(address)

{'address': '941 Spring Creek Rd',
 'city': 'Chattanooga',
 'phone': '4238947870',
 'state_code': 'TN',
 'zip_code': '37412'}

## Name & Website

In [22]:
title = location.find(class_='location-title')
title

<p class="location-title">
														<a data-open="popup" href="http://parkridgeeasthospital.com/" itemprop="name">Parkridge East Hospital</a>
														<br/>
														<span class="location-distance"> </span>
													</p>

In [23]:
link = title.find('a')
link.text

'Parkridge East Hospital'

In [24]:
link.get('href')

'http://parkridgeeasthospital.com/'

In [25]:
def extract_title(soupy_html_address):
    title = soupy_html_address.find(class_='location-title')
    link = title.find('a')
    
    name = link.text
    url = link.get('href')

    return {
        'facility': name,
        'url': url
    }

extract_title(location)

{'facility': 'Parkridge East Hospital',
 'url': 'http://parkridgeeasthospital.com/'}

### Extract a whole location

In [26]:
data = dict()
data.update(extract_address(location))
data.update(extract_title(location))
data

{'address': '941 Spring Creek Rd',
 'city': 'Chattanooga',
 'facility': 'Parkridge East Hospital',
 'phone': '4238947870',
 'state_code': 'TN',
 'url': 'http://parkridgeeasthospital.com/',
 'zip_code': '37412'}

### Grab all of them

In [28]:
facilities = []
for location in results:
    data = dict()
    data.update(extract_address(location))
    data.update(extract_title(location))
    facilities.append(data)
len(facilities)

14

In [29]:
facilities

[{'address': '941 Spring Creek Rd',
  'city': 'Chattanooga',
  'facility': 'Parkridge East Hospital',
  'phone': '4238947870',
  'state_code': 'TN',
  'url': 'http://parkridgeeasthospital.com/',
  'zip_code': '37412'},
 {'address': '2333 McCallie Ave',
  'city': 'Chattanooga',
  'facility': 'Parkridge Medical Center',
  'phone': '4236986061',
  'state_code': 'TN',
  'url': 'http://parkridgemedicalcenter.com/',
  'zip_code': '37404'},
 {'address': '1000 Tennessee 28',
  'city': 'Jasper',
  'facility': 'Parkridge West Hospital',
  'phone': '4238379500',
  'state_code': 'TN',
  'url': 'http://parkridgewesthospital.com/',
  'zip_code': '37347'},
 {'address': '2221 Murphy Ave',
  'city': 'Nashville',
  'facility': "The Children's Hospital at TriStar Centennial",
  'phone': '6153421000',
  'state_code': 'TN',
  'url': 'http://thechildrenshospitalnashville.com/',
  'zip_code': '37203'},
 {'address': '313 North Main St',
  'city': 'Ashland City',
  'facility': 'TriStar Ashland City Medical Cen